In [7]:
# note about futures contract: p = (1-d)/(u-d)

"""put option example - calculate risk neutral probability p """

# call option - in the money when K<ST can buy below market value
# put option - in the money when K>ST can sell for value higher than market value

import numpy 
import math


# u == multiplicative up factor, d == down factor
# p == risk-neutral probability 

r = 0.05
T = 2
dt = T/2
u = 1.2
d = 0.8

p = (math.exp(r*dt)-d)/(u-d)

print(p)

0.6281777409400603


In [102]:
"""writing Stockoption base class - reusable code for attributes of stock option
for this chapter"""

import math 

class StockOption(object):
    def __init__(self,S0,K,r=0.05,T=1,N=2,pu=0,pd=0,div=0,sigma=0,is_put=False, is_am=False) :
        """defaults to Euro call unless specified"""

        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.N = max(1,N) # number of time steps
        self.STs = [] # declare the stock prices tree
        
        self.pu, self.pd = pu, pd 
        self.div = div 
        self.sigma = sigma
        self.is_call = not is_put
        self.is_european = not is_am 
    
    @property
    def dt(self):
        return self.T/float(self.N)
        
        # the discount factor 
    @property
    def df(self):
        return math.exp(-(self.r-self.div)*self.dt)
        

In [103]:
"""Class for Euro options in Binomial tree"""

import math 
import numpy as np
from decimal import Decimal

class BinomialEuropeanOption(StockOption):
    def setup_parameters(self):
        self.M = self.N+1  # number of terminal nodes
        self.u = 1 + self.pu # expected value in up state
        self.d = 1-self.pd # expected value in down state
        self.qu = (math.exp((self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu
        
    def init_stock_price_tree(self):
        # initialize terminal node prices at sero
        self.STs = np.zeros(self.M)
        
        # expected stock prices at each node 
        for i in range(self.M):
            self.STs[i]=self.S0*(self.u**(self.N-i)*(self.d**i))
            
      # returns payoff when option expires at terminal node       
    def init_payoffs_tree(self):
        if self.is_call:
            return np.maximum(0, self.STs-self.K)
        else:
            return np.maximum(0, self.K-self.STs)
    
    '''starting at expiration time, traverse backwards and calculate
    discounted payoffs at each node'''
    
    # [:-1] = all elements except last, [1:] = all elements except first (0)
    def traverse_tree(self, payoffs):
        for i in range(self.N):
            payoffs = (payoffs[:-1]*self.qu + payoffs[1:]*self.qd)*self.df
            
        return payoffs
        
    def begin_tree_traversal(self):
        payoffs = self.init_payoffs_tree()
        return self.traverse_tree(payoffs)
    
    def price(self):
        """starting point of the tree and provides option value"""
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        
        return payoffs[0]

In [115]:
eu_option=BinomialEuropeanOption(50,52,r=0.05,T=2,N=2,pu=0.2,pd=0.2,is_put=False)
print(eu_option.price())
    
    

7.141108542733969


In [113]:
"""American options with binomial tree"""

import math 
import numpy as np
from decimal import Decimal

class BinomialTreeOption(StockOption):
    def setup_parameters(self):
    
        self.u = 1 + self.pu # expected value in up state
        self.d = 1-self.pd # expected value in down state
        self.qu = (math.exp((self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu
        
# store expected returns of stock price at all time steps in NumPy array

    def init_stock_price_Tree(self):
        self.STs = [np.array([self.S0])] # initialize 2D tree at T=0
        
        for i in range(self.N): # simulate possible stock prices path
            prev_branches = self.STs[-1]
            st = np.concatenate(
                (prev_branches*self.u,
                 [prev_branches[-1]*self.d]))
            self.STs.append(st) # add  nodes at each time step 

# creates payoff tree as 2D array. Starts with intrinsic option value @ maturity

    def init_payoff_tree(self):
        if self.is_call:
            return np.maximum(0,self.STs[self.N]-self.K)
        else:
            return np.maximum(0,self.K-self.STs[self.N])
  
 # returns maximum payoff values b/w exercising early or not at all
    def check_early_exercise(self,payoffs,node):
        if self.is_call:
            return np.maximum(payoffs, self.STs[node]-self.K)
        else:
            return np.maximum(payoffs,self.K-self.STs[node])
   
 # checks if optimal to exercise Am option early at every time step 
 # also moves through tree to find discount price at each step 
    def traverse_tree(self, payoffs):
        for i in reversed(range(self.N)):
            # payoff if not exercising the option
            payoffs = (payoffs[:-1]*self.qu + 
                       payoffs[1:]*self.qd)*self.df
            
            # payoff from exercising, for Am options
            if not self.is_european:
                payoffs = self.check_early_exercise(payoffs,i)
        
        return payoffs
    
    def begin_tree_traversal(self):
        payoffs = self.init_payoff_tree()
        return self.traverse_tree(payoffs)
    
    def price(self):
        """starting point of the tree and provides option value"""
        self.setup_parameters()
        self.init_stock_price_Tree()
        payoffs = self.begin_tree_traversal()
        
        return payoffs[0]

In [114]:
am_option = BinomialTreeOption(50,52,r=0.05,T=2,N=2,pu=0.2,pd=0.2,is_put=True,is_am=True)

print(am_option.price())

5.089632474198373
